first let's test things!

In [266]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = [6, 6]
%matplotlib widget
from PIL import Image
from tensorflow import keras
import warnings
from scipy import stats

In [2]:
# load the satelite images as well as the targets: the ice concentrations and types ('stage')
# separate folder for each month's data
rgb={}
conc={}
stage={}
for i in range(1,13):
    rgb[i]=np.asarray(Image.open(str(i)+'/rgb.png'))[:,:,:3] # don't need the transparency value of the png
    stage[i]=np.asarray(Image.open(str(i)+'/stage.png'))[:,:,:3]
    conc[i]=np.asarray(Image.open(str(i)+'/conc.png'))[:,:,:3]

In [541]:
plt.figure()
plt.imshow(rgb[6])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
plt.close('all')

In [540]:
plt.figure()
plt.imshow(conc[6])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
indices = np.moveaxis(np.indices(rgb[6].shape)[0:2,:,:,0], 0, 2)
allArray = np.dstack((indices, rgb[6])).reshape((-1, 5))
targetarray = np.dstack((indices, conc[6])).reshape((-1, 5))
data = pd.DataFrame(allArray, columns=['y', 'x', 'red','green','blue'])
targetDF = pd.DataFrame(targetarray, columns=['y', 'x', 'red','green','blue'])

In [36]:
print(data.shape)
print(targetDF.shape)

(8697360, 5)
(8697360, 5)


In [37]:
forbiddencolors=np.array(((135,135,135),(79,255,99),(255,255,255),(0,0,0)))
colorlabels=['cyan','green','yellow','offyellow','orange','red','white','gray']
colors=np.array(((0,233,233),(57,163,73),(255,251,7),(252,215,0),(255,128,1),(227,26,28),(255,255,255),(135,135,135)))

In [38]:
datamatch=pd.Series(np.ones(len(data), dtype=bool))
targetcolormatch=pd.Series(np.zeros(len(data), dtype=bool))
for i in range(0,len(forbiddencolors)):
    datamatch=datamatch&(~((data.loc[:,'red']==forbiddencolors[i,0])&(data.loc[:,'green']==forbiddencolors[i,1])&(data.loc[:,'blue']==forbiddencolors[i,2])))
for i in range(0,len(colors)):
    targetcolormatch=targetcolormatch|((targetDF.loc[:,'red']==colors[i,0])&(targetDF.loc[:,'green']==colors[i,1])&(targetDF.loc[:,'blue']==colors[i,2]))
fullmatch=datamatch & targetcolormatch
cleandata=data.loc[fullmatch,:]
cleantargets=targetDF.loc[fullmatch,:]

In [39]:
print(cleandata.shape)
print(cleantargets.shape)

(2159549, 5)
(2159549, 5)


In [40]:
#repopulate the picture with only the data shown
newpic=np.zeros(rgb[1].shape)
newpic[cleandata.y,cleandata.x,0]=cleandata.red
newpic[cleandata.y,cleandata.x,1]=cleandata.green
newpic[cleandata.y,cleandata.x,2]=cleandata.blue

In [41]:
plt.figure()
plt.imshow(newpic.astype(int))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
target=pd.DataFrame({'y':cleantargets.y,'x':cleantargets.x,'target':np.zeros(len(cleandata))})
for i in range(0,len(colors)):
    target.loc[(cleantargets.loc[:,'red']==colors[i,0])&(cleantargets.loc[:,'green']==colors[i,1])&(cleantargets.loc[:,'blue']==colors[i,2]),'target']=i

In [43]:
target.target.unique()

array([7., 6., 5., 2., 3., 1., 4., 0.])

In [44]:
x_train_all, x_validate, y_train_all, y_validate = model_selection.train_test_split(cleandata.loc[:,'red':'blue'], target.target, test_size=0.1)

# now split the training data further into training and testing
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train_all, y_train_all, test_size=0.2)

In [45]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
modelGNB=gnb.fit(x_train, y_train)
y_pred = modelGNB.predict(x_test)
score = accuracy_score(y_test, y_pred)
print(score)

0.7444812319438978


In [46]:
plt.figure()
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 23.52222222222222, 'predicted label')

In [47]:
target.target.value_counts()

5.0    1015395
7.0     892624
6.0     148025
4.0      43952
0.0      25780
2.0      13341
1.0      10349
3.0      10083
Name: target, dtype: int64

In [48]:
# resample the data to rebalance the classes
nsamp=np.min(target.target.value_counts())
dataresampledmat=np.zeros((nsamp,5,len(target.target.unique())))
targetresampledmat=np.zeros((nsamp,3,len(target.target.unique())))
for targ in target.target.unique():
    dataresampledmat[:,:,int(targ)],targetresampledmat[:,:,int(targ)]=resample(cleandata.loc[target.target==targ,:],target.loc[target.target==targ,:],n_samples=nsamp)
dataresampled=pd.DataFrame(np.hstack(dataresampledmat.transpose()).transpose(), columns=['y', 'x', 'red','green','blue'])
targetresampled=pd.DataFrame(np.hstack(targetresampledmat.transpose()).transpose(), columns=['y', 'x', 'target'])

In [49]:
targetresampled.target.value_counts()

0.0    10083
4.0    10083
3.0    10083
1.0    10083
7.0    10083
2.0    10083
5.0    10083
6.0    10083
Name: target, dtype: int64

In [50]:
x_train_all, x_validate, y_train_all, y_validate = model_selection.train_test_split(dataresampled.loc[:,'red':'blue'], targetresampled.target, test_size=0.1)
# now split the training data further into training and testing
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train_all, y_train_all, test_size=0.2)

In [51]:
gnb = GaussianNB()
modelGNB=gnb.fit(x_train, y_train)
y_pred = modelGNB.predict(x_test)
score = accuracy_score(y_test, y_pred)
print(score)

0.3082644628099174


In [52]:
plt.figure()
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 23.52222222222222, 'predicted label')

In [53]:
#Decision tree
from sklearn import tree
clfTree=tree.DecisionTreeClassifier()
model=clfTree.fit(x_train,y_train)
clfTree.get_depth()

37

In [54]:
# visualize the tree that was created
plt.figure(figsize=(33,5))
tree.plot_tree(model,max_depth=3,filled=True,fontsize=6,feature_names=['red','green','blue'])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
y_pred = model.predict(x_test)
score = accuracy_score(y_test, y_pred)
print(score)

0.5713498622589531


In [56]:
scores=model_selection.cross_val_score(clfTree, x_train, y_train, cv=10)
print(scores.mean(),',',scores.std())

0.5648017810982375 , 0.007231785808246094


In [57]:
from sklearn.ensemble import RandomForestClassifier
RFclf=RandomForestClassifier(max_depth=30,n_estimators=1000)
#datafilter=np.random.choice(a=[False, True], size= len(y_train), p=[0.98, 0.02])
modelRF=RFclf.fit(x_train,y_train)

In [58]:
y_pred = modelRF.predict(x_test)

In [59]:
score = accuracy_score(y_test, y_pred)
print(score)

0.5815426997245179


In [60]:
plt.figure()
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 23.52222222222222, 'predicted label')

In [61]:
colorlabels[4]

'orange'

In [163]:
testmat

array([[[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

        [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

        [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

        ...,

        [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

        [[ nan,  nan,  nan, ..., 212.,  nan,  nan],
         [ nan,  nan,  nan, ..., 255.,  nan,  nan],
         [ nan,  nan,  nan, ..., 216.,  nan,  nan]],

        [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan]]],


       [[[ nan,  nan,  nan, ...,  na

now want to combine all data and also add more features:
neighboring pixel values
mean of nearby pixels
std of nearby pixels

In [171]:
# now combine all training observations, test observation, and process validation observation
trainingN=[1,2,5,6,8,9,10]
testingN=[4]
valN=[12]
obs=trainingN.copy()
obs.extend(testingN)
obs.extend(valN)
cleandata_train_all=pd.DataFrame()
target_train_all=pd.DataFrame()
cleandata_test_all=pd.DataFrame()
target_test_all=pd.DataFrame()
cleandata_val_all=pd.DataFrame()
target_val_all=pd.DataFrame()
warnings.filterwarnings("ignore", message='Degrees of freedom <= 0 for slice.')
warnings.filterwarnings("ignore", message='Mean of empty slice')
for j in obs:
    print('Processing Image '+str(j))
    # set forbidden colors to nan
    data_nans=rgb[j].copy().astype(float)
    for i in range(0,len(forbiddencolors)):
        colormat=np.zeros(rgb[j].shape)
        colormat[:,:,:]=forbiddencolors[i,:]
        data_nans[np.sum(np.abs(colormat-rgb[j]),axis=2)==0,:]=np.nan
    #calculate nearby mean and std for each pixel
    testmat=np.zeros((rgb[j].shape[0],rgb[j].shape[1],3,25))
    pixel=0
    for x in range(-2,3):
        for y in range(-2,3):
            mat=np.roll(data_nans[:,:,0],x,axis=1)
            testmat[:,:,0,pixel]=np.roll(mat,y,axis=0)
            mat=np.roll(data_nans[:,:,1],x,axis=1)
            testmat[:,:,1,pixel]=np.roll(mat,y,axis=0)
            mat=np.roll(data_nans[:,:,2],x,axis=1)
            testmat[:,:,2,pixel]=np.roll(mat,y,axis=0)
            pixel+=1
    stdpic=np.nanstd(testmat,axis=3)
    meanpic=np.nanmean(testmat,axis=3)
    
    indices = np.moveaxis(np.indices(rgb[j].shape)[0:2,:,:,0], 0, 2)
    obsmat= j*np.ones(rgb[j].shape[:2])
    allArray = np.dstack((indices, obsmat, rgb[j], meanpic, stdpic)).reshape((-1, 12))
    targetarray = np.dstack((indices, obsmat, conc[j])).reshape((-1, 6))
    data = pd.DataFrame(allArray, columns=['y', 'x', 'obs','red','green','blue','meanR','meanG','meanB','stdR','stdG','stdB'])
    targetDF = pd.DataFrame(targetarray, columns=['y', 'x','obs', 'red','green','blue'])

    datamatch=pd.Series(np.ones(len(data), dtype=bool))
    targetcolormatch=pd.Series(np.zeros(len(data), dtype=bool))
    for i in range(0,len(forbiddencolors)):
        datamatch=datamatch&(~((data.loc[:,'red']==forbiddencolors[i,0])&(data.loc[:,'green']==forbiddencolors[i,1])&(data.loc[:,'blue']==forbiddencolors[i,2])))
    for i in range(0,len(colors)):
        targetcolormatch=targetcolormatch|((targetDF.loc[:,'red']==colors[i,0])&(targetDF.loc[:,'green']==colors[i,1])&(targetDF.loc[:,'blue']==colors[i,2]))
    fullmatch=datamatch & targetcolormatch
    cleandata=data.loc[fullmatch,:]
    cleantargets=targetDF.loc[fullmatch,:]

    target=pd.DataFrame({'y':cleandata.y,'x':cleandata.x,'obs':cleandata.obs,'target':np.zeros(len(cleandata))})
    for i in range(0,len(colors)):
        target.loc[(cleantargets.loc[:,'red']==colors[i,0])&(cleantargets.loc[:,'green']==colors[i,1])&(cleantargets.loc[:,'blue']==colors[i,2]),'target']=i
    #cleandata.drop(columns=['target_r','target_g','target_b'])
    if j in trainingN:
        cleandata_train_all=cleandata_train_all.append(cleandata)
        target_train_all=target_train_all.append(target)
    elif j in testingN:
        cleandata_test_all=cleandata_test_all.append(cleandata)
        target_test_all=target_test_all.append(target)
    elif j in valN:
        cleandata_val_all=cleandata_val_all.append(cleandata)
        target_val_all=target_val_all.append(target)


Processing Image 1
Processing Image 2
Processing Image 5
Processing Image 6
Processing Image 8
Processing Image 9
Processing Image 10
Processing Image 4
Processing Image 12


In [172]:
cleandata_train_all.head(10)

,y,x,obs,red,green,blue,meanR,meanG,meanB,stdR,stdG,stdB
1066,0.0,1066.0,1.0,127.0,153.0,130.0,115.000000,178.333333,122.333333,8.640988,18.208667,5.557777
1067,0.0,1067.0,1.0,111.0,187.0,120.0,116.750000,174.500000,123.250000,8.073878,17.109939,5.068284
1068,0.0,1068.0,1.0,107.0,195.0,117.0,118.500000,170.500000,124.333333,9.878428,21.891779,6.182412
1069,0.0,1069.0,1.0,122.0,163.0,126.0,114.500000,178.666667,121.833333,10.468206,22.954060,6.413960
1070,0.0,1070.0,1.0,134.0,135.0,134.0,115.200000,177.000000,122.200000,11.338430,24.811288,6.968501
1076,0.0,1076.0,1.0,131.0,255.0,145.0,189.857143,255.000000,197.285714,64.009884,0.000000,56.731735
1080,0.0,1080.0,1.0,159.0,255.0,170.0,217.000000,255.000000,221.333333,41.529106,0.000000,36.745370
1081,0.0,1081.0,1.0,238.0,255.0,240.0,198.500000,255.000000,205.000000,39.500000,0.000000,35.000000
3503,0.0,3503.0,1.0,212.0,255.0,216.0,202.333333,255.000000,208.333333,28.986587,0.000000,25.486380
4584,1.0,1077.0,1.0,244.0,255.0,245.0,220.500000,255.000000,224.333333,46.053411,0.000000,40.921334


In [173]:
target_train_all.head(10)

,y,x,obs,target
1066,0.0,1066.0,1.0,7.0
1067,0.0,1067.0,1.0,7.0
1068,0.0,1068.0,1.0,7.0
1069,0.0,1069.0,1.0,7.0
1070,0.0,1070.0,1.0,7.0
1076,0.0,1076.0,1.0,6.0
1080,0.0,1080.0,1.0,6.0
1081,0.0,1081.0,1.0,6.0
3503,0.0,3503.0,1.0,6.0
4584,1.0,1077.0,1.0,6.0


In [174]:
target_train_all.target.value_counts()

6.0    10132988
5.0     3347317
7.0      906477
0.0      563670
4.0      375181
1.0      225893
2.0      139161
3.0      118100
Name: target, dtype: int64

In [175]:
print(cleandata_test_all.shape)
print(target_test_all.shape)
print(cleandata_val_all.shape)
print(target_val_all.shape)

(1546329, 12)
(1546329, 4)
(2172847, 12)
(2172847, 4)


In [176]:
target_test_all.target.value_counts()

5.0    854253
6.0    528746
4.0     78048
2.0     28744
1.0     17593
3.0     17208
0.0     15703
7.0      6034
Name: target, dtype: int64

In [177]:
target_val_all.target.value_counts()

5.0    1249493
6.0     717182
4.0      80865
1.0      52771
7.0      50688
3.0      21818
0.0         30
Name: target, dtype: int64

In [178]:
target_train_all.target.value_counts().sort_values()[2]

139161

In [324]:
# resample the training observations' data to rebalance the classes
training_reduction=0.1
nsamp=int(training_reduction*(target_train_all.target.value_counts().sort_values().iloc[1]))
dataresampledmat=np.zeros((nsamp,len(cleandata_train_all.columns),len(target_train_all.target.unique())))
targetresampledmat=np.zeros((nsamp,len(target_train_all.columns),len(target_train_all.target.unique())))
for targ in target_train_all.target.unique():
    dataresampledmat[:,:,int(targ)],targetresampledmat[:,:,int(targ)]=resample(cleandata_train_all.loc[target_train_all.target==targ,:],target_train_all.loc[target_train_all.target==targ,:],n_samples=nsamp)
data_resampled=pd.DataFrame(np.hstack(dataresampledmat.transpose()).transpose(), columns=['y', 'x', 'obs', 'red','green','blue','meanR','meanG','meanB','stdR','stdG','stdB'])
target_resampled=pd.DataFrame(np.hstack(targetresampledmat.transpose()).transpose(), columns=['y', 'x', 'obs', 'target'])

# resample the test observation's data to rebalance the classes
test_reduction=0.1
nsamp=int(test_reduction*(target_test_all.target.value_counts().sort_values().iloc[1]))
dataresampledmat=np.zeros((nsamp,len(cleandata_test_all.columns),len(target_test_all.target.unique())))
targetresampledmat=np.zeros((nsamp,len(target_test_all.columns),len(target_test_all.target.unique())))
for targ in target_test_all.target.unique():
    dataresampledmat[:,:,int(targ)],targetresampledmat[:,:,int(targ)]=resample(cleandata_test_all.loc[target_test_all.target==targ,:],target_test_all.loc[target_test_all.target==targ,:],n_samples=nsamp)
test_data_resampled=pd.DataFrame(np.hstack(dataresampledmat.transpose()).transpose(), columns=['y', 'x', 'obs', 'red','green','blue','meanR','meanG','meanB','stdR','stdG','stdB'])
test_target_resampled=pd.DataFrame(np.hstack(targetresampledmat.transpose()).transpose(), columns=['y', 'x', 'obs', 'target'])

# do not rebalance the validation observation

In [325]:
data_resampled.head()

,y,x,obs,red,green,blue,meanR,meanG,meanB,stdR,stdG,stdB
0,1141.0,1073.0,8.0,240.0,4.0,2.0,232.60,4.00,1.88,21.365393,0.000000,0.587878
1,1203.0,1139.0,2.0,45.0,65.0,152.0,49.08,65.60,152.44,2.348106,4.280187,4.817302
2,765.0,1202.0,9.0,94.0,79.0,126.0,89.52,73.04,130.24,3.634501,4.303301,7.436558
3,1978.0,2131.0,6.0,252.0,5.0,2.0,253.80,5.00,2.00,1.720465,0.000000,0.000000
4,781.0,1439.0,9.0,157.0,91.0,113.0,147.52,91.20,108.20,13.139619,5.973274,5.775812


In [326]:
target_resampled.head()

,y,x,obs,target
0,1141.0,1073.0,8.0,0.0
1,1203.0,1139.0,2.0,0.0
2,765.0,1202.0,9.0,0.0
3,1978.0,2131.0,6.0,0.0
4,781.0,1439.0,9.0,0.0


In [327]:
target_resampled.target.value_counts()

0.0    13916
4.0    13916
3.0    13916
1.0    13916
7.0    13916
2.0    13916
5.0    13916
6.0    13916
Name: target, dtype: int64

In [328]:
x_train_all, x_validate, y_train_all, y_validate = model_selection.train_test_split(data_resampled.loc[:,'red':'stdB'], target_resampled.target, test_size=0.1)
# now split the training data further into training and testing
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train_all, y_train_all, test_size=0.2)
# now split the training data further for plotting and use when the fitting runs slowly
x_train_small, x_test_small, y_train_small, y_test_small = model_selection.train_test_split(x_train_all, y_train_all, test_size=0.02,train_size=0.02)

In [329]:
y_train_small.value_counts()

6.0    268
3.0    262
2.0    252
5.0    250
7.0    245
1.0    243
0.0    242
4.0    241
Name: target, dtype: int64

In [330]:
plt.close('all')

In [331]:
x_train_small

,red,green,blue,meanR,meanG,meanB,stdR,stdG,stdB
25156,90.0,30.0,28.0,99.520000,34.520000,38.760000,15.323498,16.062677,20.702232
75741,101.0,50.0,49.0,128.041667,32.916667,33.458333,61.607953,14.820922,19.384227
4229,253.0,13.0,5.0,239.240000,13.000000,5.000000,7.834692,0.632456,0.000000
45279,104.0,62.0,67.0,118.920000,59.720000,64.120000,13.114633,4.303673,5.757222
31253,106.0,18.0,20.0,96.960000,15.600000,20.440000,15.805012,4.326662,7.960302
...,...,...,...,...,...,...,...,...,...
38929,118.0,74.0,76.0,105.160000,54.760000,63.760000,12.177619,11.856745,9.957028
25146,92.0,24.0,23.0,151.260870,21.565217,23.478261,47.161349,2.618099,5.531487
48528,87.0,39.0,51.0,96.920000,40.080000,48.840000,10.691754,4.938988,4.144201
42909,101.0,90.0,103.0,108.160000,68.360000,80.840000,10.429497,15.354166,21.118106


In [332]:
forpariplot=x_train_small.copy()
forpariplot['target']=y_train_small
sns.pairplot(forpariplot.loc[np.isin(forpariplot['target'],(5,6,7))], hue="target")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [333]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=2)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=1, loss="hinge",max_iter=10000,tol=0.01))
    ])

polynomial_svm_clf.fit(x_train_small, y_train_small)

C:\Users\jms\anaconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('poly_features', PolynomialFeatures()),
                ('scaler', StandardScaler()),
                ('svm_clf',
                 LinearSVC(C=1, loss='hinge', max_iter=10000, tol=0.01))])

In [334]:
y_pred=polynomial_svm_clf.predict(x_test)

In [335]:
score = accuracy_score(y_test, y_pred)
print(score)

0.44128948550326863


In [336]:
plt.figure()
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [337]:
# try it on the test observations
y_pred = polynomial_svm_clf.predict(test_data_resampled.loc[:,'red':'stdB'])
score = accuracy_score(test_target_resampled.target, y_pred)
print(score)

0.37977707006369427


In [338]:
RFclf=RandomForestClassifier(max_depth=10,n_estimators=500)
modelRF=RFclf.fit(x_train,y_train)
y_pred = modelRF.predict(x_test)
score = accuracy_score(y_test, y_pred)
print(score)

0.5978841259543889


In [339]:
plt.figure()
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [340]:
#print importance of the features
for name, score in zip(list(x_train.columns), RFclf.feature_importances_):
    print(name,': ', score)

red :  0.03617637753910158
green :  0.1263806937016087
blue :  0.06943931382887873
meanR :  0.08613166731708104
meanG :  0.19561756394967958
meanB :  0.1256287782495865
stdR :  0.12568664171561167
stdG :  0.15905616646561443
stdB :  0.07588279723283789


In [341]:
y_pred = modelRF.predict(test_data_resampled.loc[:,'red':'stdB'])
score = accuracy_score(test_target_resampled.target, y_pred)
print(score)

0.37563694267515924


In [342]:
plt.figure()
mat = confusion_matrix(test_target_resampled.target, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [343]:
modelNN = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu",input_shape=(9,)),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(8, activation="softmax")
])
modelNN.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.002, momentum=0.8, nesterov=True),
              metrics=["accuracy"])

In [344]:
modelNN.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_26 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_27 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_28 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_29 (Dense)             (None, 8)                 808       
Total params: 42,208
Trainable params: 42,208
Non-trainable params: 0
__________________________________________________

In [345]:
history = modelNN.fit(x_train, y_train, epochs=20, validation_data=(x_test_small, y_test_small))

Epoch 1/20
2505/2505 [==============================] - 3s 1ms/step - loss: 1.3547 - accuracy: 0.4943 - val_loss: 1.1868 - val_accuracy: 0.5324
Epoch 2/20
2505/2505 [==============================] - 3s 1ms/step - loss: 1.1748 - accuracy: 0.5492 - val_loss: 1.1785 - val_accuracy: 0.5449
Epoch 3/20
2505/2505 [==============================] - 3s 1ms/step - loss: 1.1220 - accuracy: 0.5711 - val_loss: 1.1209 - val_accuracy: 0.5629
Epoch 4/20
2505/2505 [==============================] - 3s 1ms/step - loss: 1.0870 - accuracy: 0.5801 - val_loss: 1.1093 - val_accuracy: 0.5729
Epoch 5/20
2505/2505 [==============================] - 3s 1ms/step - loss: 1.0634 - accuracy: 0.5926 - val_loss: 1.0309 - val_accuracy: 0.6053
Epoch 6/20
2505/2505 [==============================] - 3s 1ms/step - loss: 1.0442 - accuracy: 0.5974 - val_loss: 0.9945 - val_accuracy: 0.6043
Epoch 7/20
2505/2505 [==============================] - 3s 1ms/step - loss: 1.0294 - accuracy: 0.6043 - val_loss: 1.0258 - val_accuracy:

In [346]:
#test the NN on the full test subset
y_pred = modelNN.predict(x_test)
y_pred=np.argmax(y_pred,axis=1)
score = accuracy_score(y_test, y_pred)
print(score)

0.633764159888218


In [347]:
#test the NN on the test observation
y_pred = modelNN.predict(test_data_resampled.loc[:,'red':'stdB'])
y_pred=np.argmax(y_pred,axis=1)
score = accuracy_score(test_target_resampled.target, y_pred)
print(score)

0.3393312101910828


In [348]:
plt.figure()
mat = confusion_matrix(test_target_resampled.target, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [221]:
#show the data for the test observation(s)
newpic=np.zeros(rgb[1].shape)
data4plot=cleandata_test_all.loc[cleandata_test_all.obs==testingN[0]]
newpic[data4plot.y.astype(int),data4plot.x.astype(int),0]=data4plot.red
newpic[data4plot.y.astype(int),data4plot.x.astype(int),1]=data4plot.green
newpic[data4plot.y.astype(int),data4plot.x.astype(int),2]=data4plot.blue
plt.figure()
plt.imshow(newpic.astype(int))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

step back: try to figure out how many components I should be able to expect to successfully capture

In [225]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.fit_transform(x_test)

In [227]:
from sklearn.cluster import KMeans
intertias=np.zeros(8)
for i in range(0,8):
    print('Starting '+str(i))
    clf=KMeans(n_clusters=i+1)
    model=clf.fit(x_train_scaled)
    intertias[i]=model.inertia_
plt.figure()
plt.plot(np.arange(8)+1,intertias)

Starting 0
Starting 1
Starting 2
Starting 3
Starting 4
Starting 5
Starting 6
Starting 7


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [228]:
from sklearn.metrics import silhouette_score
scores=np.zeros(8)
for i in range(1,8):
    print('Starting '+str(i))
    clf=KMeans(n_clusters=i+1)
    model=clf.fit(x_train_scaled)
    y_pred=model.predict(x_test_scaled)
    scores[i]=silhouette_score(x_test_scaled,y_pred)
plt.figure()
plt.plot(np.arange(8)+1,scores)

Starting 1
Starting 2
Starting 3
Starting 4
Starting 5
Starting 6
Starting 7


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [247]:
#show distribution of the clusters
clfKM=KMeans(n_clusters=2)
model=clfKM.fit(x_train_scaled)
data4plot=cleandata_train_all.loc[cleandata_train_all.obs==trainingN[3]]
y_pred=model.predict(scaler.fit_transform(data4plot.loc[:,'red':'stdB']))
newpic=np.zeros(rgb[1].shape[:2])-1
newpic[data4plot.y.astype(int),data4plot.x.astype(int)]=y_pred
plt.figure()
plt.imshow(newpic.astype(int))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [232]:
from sklearn.mixture import GaussianMixture
bics=np.zeros(8)
aics=np.zeros(8)
for i in range(0,8):
    print('Starting '+str(i))
    clf=GaussianMixture(n_components=i+1)
    model=clf.fit(x_train)
    bics[i]=model.bic(x_train)
    aics[i]=model.aic(x_train)
plt.figure()
plt.plot(np.arange(8)+1,aics)
plt.plot(np.arange(8)+1,bics)

Starting 0
Starting 1
Starting 2
Starting 3
Starting 4
Starting 5
Starting 6
Starting 7


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [444]:
#reconstruct the pictures to use in a CNN, then chop them into 11X11 squares for passing to the CNN 
subimagesize=13
step=13
nx=int(rgb[1].shape[1]/step)
ny=int(rgb[1].shape[0]/step)
subimagesTrain=np.zeros((nx*ny*len(trainingN),subimagesize,subimagesize,3))
subimgtargetTrain=np.zeros(nx*ny*len(trainingN))-100
subimagesTest=np.zeros((nx*ny*len(testingN),subimagesize,subimagesize,3))
subimgtargetTest=np.zeros(nx*ny*len(testingN))-100
subimagesVal=np.zeros((nx*ny*len(valN),subimagesize,subimagesize,3))
subimgtargetVal=np.zeros(nx*ny*len(valN))-100
train_i=0
test_i=0
val_i=0
for j in obs:
    print('Starting obs '+str(j))
    if j in trainingN:
        data=cleandata_train_all.copy()
        targs=target_train_all.copy()
        train_i+=1
    elif j in testingN:
        data=cleandata_test_all.copy()
        targs=target_test_all.copy()
        test_i+=1
    elif j in valN:
        data=cleandata_val_all.copy()
        targs=target_val_all.copy()
        val_i+=1
    newpic=np.zeros(rgb[j].shape)
    newtargpic=np.zeros(rgb[j].shape[:2])-100
    data4plot=data.loc[data.obs==j]
    targs4plot=targs.loc[targs.obs==j]
    newpic[data4plot.y.astype(int),data4plot.x.astype(int),0]=data4plot.red
    newpic[data4plot.y.astype(int),data4plot.x.astype(int),1]=data4plot.green
    newpic[data4plot.y.astype(int),data4plot.x.astype(int),2]=data4plot.blue
    newtargpic[targs4plot.y.astype(int),targs4plot.x.astype(int)]=targs4plot.target
    
    # now chop the images into squares
    startx=0
    endx=startx+subimagesize
    for i in range(0,nx):
        starty=0
        endy=starty+subimagesize
        for k in range(0,ny):
            #print(i,k,stats.mode(newtargpic[starty:endy,startx:endx],axis=None))
            modetarget=int(stats.mode(newtargpic[starty:endy,startx:endx],axis=None)[0])
            if np.all(newpic[starty:endy,startx:endx,:]==0)|(modetarget==-100):
                starty=starty+step
                endy=starty+subimagesize
                continue  #skip bad data
            if j in trainingN:
                subimagesTrain[(train_i-1)*nx*ny+ny*i+k,:,:,:]=newpic[starty:endy,startx:endx,:]
                subimgtargetTrain[(train_i-1)*nx*ny+ny*i+k]=modetarget
            elif j in testingN:
                subimagesTest[(test_i-1)*nx*ny+ny*i+k,:,:,:]=newpic[starty:endy,startx:endx,:]
                subimgtargetTest[(test_i-1)*nx*ny+ny*i+k]=modetarget
            elif j in valN:
                subimagesVal[(val_i-1)*nx*ny+ny*i+k,:,:,:]=newpic[starty:endy,startx:endx,:]
                subimgtargetVal[(val_i-1)*nx*ny+ny*i+k]=modetarget
            starty=starty+step
            endy=starty+subimagesize
        startx=startx+step
        endx=startx+subimagesize
subimagesTrain=subimagesTrain[subimgtargetTrain!=-100,:,:,:]
subimgtargetTrain=subimgtargetTrain[subimgtargetTrain!=-100]      
subimagesTest=subimagesTest[subimgtargetTest!=-100,:,:,:]
subimgtargetTest=subimgtargetTest[subimgtargetTest!=-100]  
subimagesVal=subimagesVal[subimgtargetVal!=-100,:,:,:]
subimgtargetVal=subimgtargetVal[subimgtargetVal!=-100]  
print('Done!')

Starting obs 1
Starting obs 2
Starting obs 5
Starting obs 6
Starting obs 8
Starting obs 9
Starting obs 10
Starting obs 4
Starting obs 12
Done!


In [445]:
print(subimagesTrain.shape)
print(subimgtargetTrain.shape)
print(subimagesTest.shape)
print(subimgtargetTest.shape)
print(subimagesVal.shape)
print(subimgtargetVal.shape)

(95453, 13, 13, 3)
(95453,)
(9550, 13, 13, 3)
(9550,)
(13267, 13, 13, 3)
(13267,)


In [446]:
pd.Series(subimgtargetTrain).value_counts()

6.0    59954
5.0    21286
7.0     5393
0.0     3497
4.0     2329
1.0     1392
2.0      873
3.0      729
dtype: int64

In [447]:
pd.Series(subimgtargetTest).value_counts()

5.0    5553
6.0    3068
4.0     497
2.0     170
3.0     100
1.0      92
0.0      70
dtype: int64

In [448]:
pd.Series(subimgtargetVal).value_counts()

5.0    7903
6.0    4153
4.0     492
1.0     328
7.0     261
3.0     130
dtype: int64

In [449]:
# resample the training observations' data to rebalance the classes
training_reduction=1
subimgtargetTrainDF=pd.Series(subimgtargetTrain)
nsamp=int(training_reduction*(subimgtargetTrainDF.value_counts().sort_values().iloc[1]))
dataresampledmat=np.zeros((nsamp,subimagesize,subimagesize,3,len(subimgtargetTrainDF.unique())))
targetresampledmat=np.zeros((nsamp,len(subimgtargetTrainDF.unique())))
for targ in subimgtargetTrainDF.unique():
    dataresampledmat[:,:,:,:,int(targ)],targetresampledmat[:,int(targ)]=resample(subimagesTrain[subimgtargetTrain==targ,:,:,:],subimgtargetTrain[subimgtargetTrain==targ],n_samples=nsamp)
data_img_resampled=np.vstack(np.moveaxis(dataresampledmat, -1, 0))
targ_img_resampled=np.hstack(targetresampledmat.transpose())


# resample the test observation's data to rebalance the classes
testing_reduction=1
subimgtargetTestDF=pd.Series(subimgtargetTest)
nsamp=int(testing_reduction*(subimgtargetTestDF.value_counts().sort_values().iloc[1]))
dataresampledmat=np.zeros((nsamp,subimagesize,subimagesize,3,len(subimgtargetTestDF.unique())))
targetresampledmat=np.zeros((nsamp,len(subimgtargetTestDF.unique())))
for targ in subimgtargetTestDF.unique():
    dataresampledmat[:,:,:,:,int(targ)],targetresampledmat[:,int(targ)]=resample(subimagesTest[subimgtargetTest==targ,:,:,:],subimgtargetTest[subimgtargetTest==targ],n_samples=nsamp)
data_test_img_resampled=np.vstack(np.moveaxis(dataresampledmat, -1, 0))
targ_test_img_resampled=np.hstack(targetresampledmat.transpose())


# do not rebalance the validation observation

In [450]:
print(data_img_resampled.shape)
pd.Series(targ_img_resampled).value_counts()

(6984, 13, 13, 3)


0.0    873
6.0    873
5.0    873
1.0    873
4.0    873
2.0    873
3.0    873
7.0    873
dtype: int64

In [451]:
print(data_test_img_resampled.shape)
pd.Series(targ_test_img_resampled).value_counts()

(644, 13, 13, 3)


0.0    92
3.0    92
2.0    92
5.0    92
1.0    92
4.0    92
6.0    92
dtype: int64

In [492]:
plt.figure()
plt.imshow(x_train[1672,:,:,:].astype(int))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [452]:
x_train_all, x_validate, y_train_all, y_validate = model_selection.train_test_split(data_img_resampled, targ_img_resampled, test_size=0.1)
# now split the training data further into training and testing
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train_all, y_train_all, test_size=0.2)
# now split the training data further for plotting and use when the fitting runs slowly
#x_train_small, x_test_small, y_train_small, y_test_small = model_selection.train_test_split(x_train_all, y_train_all, test_size=0.02,train_size=0.02)

In [453]:
modelCNN = keras.models.Sequential([
    keras.layers.Conv2D(20, 7, activation="selu", padding="same",
                        input_shape=x_train[0,:,:,:].shape),
    keras.layers.Conv2D(100, 5, activation="selu", padding="same"),
    keras.layers.Conv2D(100, 5, activation="selu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(120, 3, activation="selu", padding="same"),
    keras.layers.Conv2D(120, 3, activation="selu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(40, activation="selu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8, activation="softmax")
])

In [454]:
modelCNN.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 13, 13, 20)        2960      
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 13, 13, 100)       50100     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 13, 13, 100)       250100    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 6, 6, 100)         0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 6, 6, 120)         108120    
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 6, 6, 120)         129720    
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 3, 3, 120)       

In [455]:
modelCNN.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.02),
              metrics=["accuracy"])

In [456]:
history = modelCNN.fit(x_train/ 255.0, y_train, epochs=20,
                     validation_data=(x_test/255., y_test))

Epoch 1/20
158/158 [==============================] - 26s 168ms/step - loss: 3.0536 - accuracy: 0.1784 - val_loss: 2.0768 - val_accuracy: 0.1854
Epoch 2/20
158/158 [==============================] - 28s 175ms/step - loss: 1.8288 - accuracy: 0.2967 - val_loss: 1.9358 - val_accuracy: 0.2593
Epoch 3/20
158/158 [==============================] - 28s 177ms/step - loss: 1.6849 - accuracy: 0.3228 - val_loss: 3.0083 - val_accuracy: 0.1822
Epoch 4/20
158/158 [==============================] - 28s 180ms/step - loss: 1.6533 - accuracy: 0.3620 - val_loss: 1.9340 - val_accuracy: 0.2721
Epoch 5/20
158/158 [==============================] - 31s 196ms/step - loss: 1.5685 - accuracy: 0.3978 - val_loss: 1.5285 - val_accuracy: 0.4177
Epoch 6/20
158/158 [==============================] - 29s 182ms/step - loss: 1.5189 - accuracy: 0.4113 - val_loss: 2.3145 - val_accuracy: 0.2498
Epoch 7/20
158/158 [==============================] - 29s 181ms/step - loss: 1.4701 - accuracy: 0.4362 - val_loss: 2.4200 - val_ac

In [457]:
modelCNN.evaluate(x_test/255., y_test)

40/40 [==============================] - 2s 42ms/step - loss: 1.3315 - accuracy: 0.5744


[1.3314628601074219, 0.5743834376335144]

In [462]:
#test the CNN on the test observation
y_pred = modelCNN.predict(data_test_img_resampled/255.)
y_pred=np.argmax(y_pred,axis=1)
score = accuracy_score(targ_test_img_resampled, y_pred)
print(score)

0.2717391304347826


In [463]:
plt.figure()
mat = confusion_matrix(targ_test_img_resampled, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [470]:
x4RF=np.reshape(x_train,(len(x_train),-1))
x4RFtest=np.reshape(x_test,(len(x_test),-1))

In [521]:
RFclf=RandomForestClassifier(max_depth=15,n_estimators=500,verbose=0)
modelRF=RFclf.fit(x4RF,y_train)
y_pred = modelRF.predict(x4RFtest)
score = accuracy_score(y_test, y_pred)
print(score)

0.7008750994431185


In [522]:
x4RFtest2=np.reshape(data_test_img_resampled,(len(data_test_img_resampled),-1))
y_pred = modelRF.predict(x4RFtest2)
score = accuracy_score(targ_test_img_resampled, y_pred)
print(score)

0.28726708074534163


In [494]:
colorlabels[6]

'white'

In [525]:
y_train_reduced=y_train.copy()
y_train_reduced[(y_train_reduced<5)]=5
y_test_reduced=y_test.copy()
y_test_reduced[(y_test_reduced<5)]=5
y_test_reduced2=targ_test_img_resampled.copy()
y_test_reduced2[(y_test_reduced2<5)]=5
y_val_reduced=y_validate.copy()
y_val_reduced[(y_val_reduced<5)]=5

In [515]:
history = modelCNN.fit(x_train/ 255.0, y_train_reduced, epochs=4,
                     validation_data=(x_test/255., y_test_reduced))

Epoch 1/2
158/158 [==============================] - 31s 194ms/step - loss: 0.4485 - accuracy: 0.8829 - val_loss: 0.3140 - val_accuracy: 0.8918
Epoch 2/2
158/158 [==============================] - 30s 189ms/step - loss: 0.1426 - accuracy: 0.9475 - val_loss: 0.2785 - val_accuracy: 0.9244


In [516]:
#test the CNN on the test observation
y_pred = modelCNN.predict(data_test_img_resampled/255.)
y_pred=np.argmax(y_pred,axis=1)
score = accuracy_score(y_test_reduced2, y_pred)
print(score)

0.9254658385093167


In [517]:
plt.figure()
mat = confusion_matrix(y_test_reduced2, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [527]:
RFclf=RandomForestClassifier(max_depth=15,n_estimators=500,verbose=0)
modelRF=RFclf.fit(x4RF,y_train_reduced)
y_pred = modelRF.predict(x4RFtest)
score = accuracy_score(y_test_reduced, y_pred)
print(score)

0.9220365950676214


In [528]:
x4RFtest2=np.reshape(data_test_img_resampled,(len(data_test_img_resampled),-1))
y_pred = modelRF.predict(x4RFtest2)
score = accuracy_score(y_test_reduced2, y_pred)
print(score)

0.953416149068323


In [529]:
plt.figure()
mat = confusion_matrix(y_test_reduced2, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [531]:
x4RFval=np.reshape(x_validate,(len(x_validate),-1))
y_pred = modelRF.predict(x4RFval)
score = accuracy_score(y_val_reduced, y_pred)
print(score)

0.9084406294706724


In [533]:
subimagesVal.shape

(13267, 13, 13, 3)

In [536]:
y_val_reduced2=subimgtargetVal.copy()
y_val_reduced2[(y_val_reduced2<5)]=5

In [537]:
#test the RF on the val observation
x4RFval2=np.reshape(subimagesVal,(len(subimagesVal),-1))
y_pred = modelRF.predict(x4RFval2)
score = accuracy_score(y_val_reduced2, y_pred)
print(score)

0.9467852566518429


In [538]:
plt.figure()
mat = confusion_matrix(y_val_reduced2, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [542]:
plt.figure()
plt.imshow(rgb[12])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [543]:
plt.figure()
plt.imshow(conc[12])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …